## Quick and dirty trust simulation

In [ ]:
import mate
import avstack

from avstack.datastructs import DataContainer
from avstack.geometry import GlobalOrigin2D, Position, Circle
from avstack.modules.perception.detections import CentroidDetection
from avstack.modules.tracking.tracker2d import BasicXyTracker

import numpy as np
import matplotlib.pyplot as plt
from labellines import labelLine, labelLines

import os

TINY_SIZE = 10
SMALL_SIZE = 14
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=TINY_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

marker_large = 10
marker_medium = 8
marker_small = 5
linewidth = 3

fig_dir = "figures"
os.makedirs(fig_dir, exist_ok=True)

In [ ]:
def init_agents(radius):
    # agent info
    agents = [
        np.array([0, 0]),
        np.array([1, 0]),
        np.array([1/2, np.sqrt(3/4)])
    ]
    fovs = [Circle(radius=radius, center=agent) for agent in agents]
    platforms = [GlobalOrigin2D for _ in agents]

    return agents, fovs, platforms


def init_objects(fovs, n_objects=5, min_dist=0.1, force_object_case=False):
    if force_object_case: 
        objects = [
            np.array([0, 0.5]),   # in agents 0, 1
            np.array([0.2, 0.5]), # in agents 0, 1
            np.array([-0.5, 0]),  # in agent  1
            np.array([0.5, 0.5]), # in agents 0, 1, 2
            np.array([1, 0.6]),   # in agents 1, 2
        ]
    else:
        objects = []
        for _ in range(n_objects):
            while True:
                obj = np.random.uniform(low=-0.5, high=1.5, size=(2,))
                if any(fov.check_point(obj) for fov in fovs):
                    if not any(np.linalg.norm(obj - o2) < min_dist for o2 in objects):
                        break
            objects.append(obj)
    return objects


def init_adversary(fovs):
    # false positives
    fps = [
        (0, np.array([0.5, 0.3])),  # (agent 0, location)
        (0, np.array([0.45, -0.5])),
        (1, np.array([0.8, 0.4])),
    ]

    # false negatives
    fns = [
        (0, 1),  # (agent 0, object 1)
    ]
    return fps, fns    


def init_tracker():
    tracker = avstack.modules.tracking.multisensor.MeasurementBasedMultiTracker(
        tracker=BasicXyTracker(threshold_confirmed=4),
        platform=GlobalOrigin2D,
    )
    return tracker


def detect(agents, radius, objects, fps, fns, frame, timestamp):
    detections = {}
    labels = {}
    for i, agent in enumerate(agents):
        dets = []
        for j, obj in enumerate(objects):
            if (i,j) in fns:
                continue
            else:
                if np.linalg.norm(agent - obj) <= radius:
                    dets.append(CentroidDetection(i, obj, reference=GlobalOrigin2D))
        for j, fp in enumerate(fps):
            if fp[0] == i:
                dets.append(CentroidDetection(i, fp[1], reference=GlobalOrigin2D))
        detections[i] = DataContainer(frame=frame, timestamp=timestamp, data=dets, source_identifier=i)
    return detections


def plot_agents_objects(agents, radius, objects, fps, fns):
    # -- plot full picture
    fig, ax = plt.subplots()
    agent_colors = "rgb"
    # plot agents
    for agent, color in zip(agents, agent_colors):
        ax.plot(*agent, '*', markersize=marker_medium, color=color)
        circle = plt.Circle(agent, radius, color=color, alpha=0.4)
        ax.add_patch(circle)
    
    # plot objects
    for i, object in enumerate(objects):
        ax.plot(*object, 'o', markersize=marker_medium, color="black", label="Truth" if i==0 else "")
    
    # plot false positives
    for i, fp in enumerate(fps):
        ax.plot(*fp[1], 'x', markersize=marker_medium, color=agent_colors[fp[0]], label="FP" if i == 0 else "")
    
    # plot false negatives
    for i, fn in enumerate(fns):
        ax.plot(*objects[fn[1]], '+', markersize=marker_large, color=agent_colors[fn[0]], label="FN" if i == 0 else "")
    
    # set limits
    ax.set_xlim([-1, 2])
    ax.set_ylim([-1, 1.8])
    ax.set_aspect('equal', adjustable='box')
    plt.legend()
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, "experiment_truth.pdf"))
    plt.show()
    

def plot_detections(agents, radius, dets, objects=None):
    # -- plot detections only
    fig, ax = plt.subplots()
    agent_colors = "rgb"
    # plot agents
    for agent, color in zip(agents, agent_colors):
        ax.plot(*agent, '*', markersize=marker_large, color=color)
        circle = plt.Circle(agent, radius, color=color, alpha=0.4)
        ax.add_patch(circle)
    
    # plot objects
    if objects:
        for i, object in enumerate(objects):
            ax.plot(*object, 'o', markersize=marker_medium, color="black", label="Truth" if i==0 else "")
    
    # plot detections
    det_markers = "123"
    for i_agent, ds in dets.items():
        for j, det in enumerate(ds):
            ax.plot(*det.x, det_markers[i_agent], markersize=marker_large, alpha=1, color=agent_colors[i_agent],
                    label="Detection" if j==0 else "")
    
    # set limits
    ax.set_xlim([-1, 2])
    ax.set_ylim([-1, 1.8])
    ax.set_aspect('equal', adjustable='box')
    plt.legend()
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, "experiment_detections.pdf"))
    plt.show()


def plot_clusters(agents, radius, clusters, objects=None):
    # plot clusters
    fig, ax = plt.subplots()
    agent_colors = "rgb"
    # plot agents
    for agent, color in zip(agents, agent_colors):
        ax.plot(*agent, '*', markersize=marker_large, color=color)
        circle = plt.Circle(agent, radius, color=color, alpha=0.4)
        ax.add_patch(circle)
    
    # plot objects
    if objects:
        for i, object in enumerate(objects):
            ax.plot(*object, 'o', markersize=marker_large, color="black", label="Truth" if i==0 else "")
    
    # plot clusters
    for j, clust in enumerate(clusters):
        pos = clust.centroid().x
        ax.plot(*pos, "x", markersize=marker_large, color="orange", label="Cluster" if j==0 else "", alpha=0.8)
        ax.text(pos[0]+0.02, pos[1]+0.02, j)
    
    # set limits
    ax.set_xlim([-1, 2])
    ax.set_ylim([-1, 1.8])
    ax.set_aspect('equal', adjustable='box')
    plt.legend()
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("figures/experiment_clusters.pdf")
    plt.show()


def plot_tracks(agents, radius, tracks, objects=None):
    # plot tracks
    fig, ax = plt.subplots()
    agent_colors = "rgb"
    # plot agents
    for agent, color in zip(agents, agent_colors):
        ax.plot(*agent, '*', markersize=marker_large, color=color)
        circle = plt.Circle(agent, radius, color=color, alpha=0.4)
        ax.add_patch(circle)
    
    # plot objects
    if objects:
        for i, object in enumerate(objects):
            ax.plot(*object, 'o', markersize=marker_large, color="black", label="Truth" if i==0 else "")
    
    # plot tracks
    for j, track in enumerate(tracks):
        pos = track.position.x
        ax.plot(*pos, "x", markersize=marker_large, color="orange", label="Track" if j==0 else "", alpha=0.8)
        ax.text(pos[0]+0.02, pos[1]+0.02, j)
    
    # set limits
    ax.set_xlim([-1, 2])
    ax.set_ylim([-1, 1.8])
    ax.set_aspect('equal', adjustable='box')
    plt.legend()
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("figures/experiment_tracks.pdf")
    plt.show()

In [ ]:
class PSM:
    def __init__(self, result, dist=None, p_exist=None):
        if result:
            assert dist is not None
        self.result = result
        self.dist = dist
        self.p_exist = p_exist

    def __repr__(self):
        return self.__str__()

    def __str__(self):
        return f"Pseudomeasurement: ({self.value}, {self.confidence})"

    @property
    def value(self):
        return self.result
    
    @property
    def confidence(self):
        return self.p_exist


class TrustEstimator:
    def __init__(self, assign_radius=0.05, delta_m_factor=50, delta_v_factor=0.05, do_propagate=True, propagation_model="uncertainty"):
        self.clusterer = avstack.modules.clustering.SampledAssignmentClusterer(assign_radius=assign_radius)
        self.delta_m_factor = delta_m_factor
        self.delta_v_factor = delta_v_factor
        self.do_propagate = do_propagate
        self._propagation_model = propagation_model
        self._init_params = {"alpha": 0.5, "beta": 0.5}
        self.params = {}
        self.tracks = {}

    def __call__(self, agents, fovs, dets, tracks):
        if self.do_propagate:
            self.propagate()
        clusters = self.update(agents, fovs, dets, tracks)
        return clusters

    @staticmethod
    def psm(agents, fovs, cluster, track):
        psms = []
        for i_agent, (agent, fov) in enumerate(zip(agents, fovs)):
            if fov.check_point(cluster.centroid().x):
                # Get the PSM
                saw = i_agent in cluster.agent_IDs  # did we see it?
                if saw:  # positive result
                    dets = cluster.get_objects_by_agent_ID(i_agent)
                    if len(dets) > 1:
                        raise RuntimeError
                    dist = cluster.distance(dets[0])
                    psm = PSM(1.0, dist=dist, p_exist=track.probability)
                else:  # negative result
                    psm = PSM(0.0, p_exist=1.0)
                psms.append(psm)
            else:
                pass  # not expected to see
        return psms
    
    def propagate(self):
        # Each frame we add some uncertainty
        for ID in self.params:
            if self._propagation_model == "uncertainty":
                # add variance
                a = self.params[ID]["alpha"]
                b = self.params[ID]["beta"]
                m = a / (a + b)
                v = a + b
                m += (0.5 - m)/self.delta_m_factor
                v += self.delta_v_factor
                self.params[ID]["alpha"] = m * v
                self.params[ID]["beta"]  = (1-m) * v
            elif self._propagation_model == "prior":
                # move closer to the prior parameters
                w = 1/2
                self.params[ID]["alpha"] = w*self.params[ID]["alpha"] + (1-w)*self._init_params["alpha"]
                self.params[ID]["beta"]  = w*self.params[ID]["beta"]  + (1-w)*self._init_params["beta"]
            elif self._propagation_model == "normalize":
                # perform a heuristic normalization on params
                s = (self.params[ID]["alpha"] + self.params[ID]["beta"])/2
                self.params[ID]["alpha"] /= s
                self.params[ID]["beta"] /= s
            else:
                raise NotImplementedError

    def update(self, agents, fovs, dets, tracks):
        # save the tracks
        self.tracks = tracks
        for track in tracks:
            if track.ID not in self.params:
                self.params[track.ID] = self._init_params.copy()
        
        # cluster the detections
        clusters = self.clusterer(dets, frame=0, timestamp=0)
        
        # assign clusters to existing tracks for IDs
        A = avstack.modules.assignment.build_A_from_distance(clusters, tracks)
        assign = avstack.modules.assignment.gnn_single_frame_assign(A, cost_threshold=0.2)
        
        # assignments - run pseudomeasurement generation
        for j_clust, i_track in assign.iterate_over("rows", with_cost=False).items():
            i_track = i_track[0]  # one edge only
            ID_track = tracks[i_track].ID
            psms = self.psm(agents, fovs, clusters[j_clust], tracks[i_track]) 
            
            # update the parameters
            if len(psms) > 1:
                for psm in psms:
                    self.params[ID_track]["alpha"] += psm.confidence * psm.value
                    self.params[ID_track]["beta"]  += psm.confidence * (1 - psm.value)

        # lone clusters - do not do anything, assume they start new tracks
        if len(assign.unassigned_rows) > 0:
            pass

        # lone tracks - penalize because of no detections (if in view)
        if len(assign.unassigned_cols) > 0:
            for i_track in assign.unassigned_cols:
                ID_track = tracks[i_track].ID
                psm = PSM(0.0, p_exist=1.0)  # TODO: merge this in with other PSM generation
                self.params[ID_track]["alpha"] += psm.confidence * psm.value
                self.params[ID_track]["beta"]  += psm.confidence * (1 - psm.value)
        
        return clusters


def plot_trust(trust_estimator, objects=None):
    # assign last tracks to truths, if possible
    if objects is not None:
        A = avstack.modules.assignment.build_A_from_distance(trust_estimator.tracks, objects)
        assigns = avstack.modules.assignment.gnn_single_frame_assign(A, cost_threshold=0.1)
    else:
        assigns = None

    # plot all trust distributions
    from scipy.stats import beta
    fig, ax = plt.subplots()
    x = np.linspace(0, 1.0, 10000)
    IDs = np.array([trk.ID for trk in trust_estimator.tracks])
    for ID in sorted(trust_estimator.params):
        idx_track = np.argwhere(ID == IDs)[0][0]
        if assigns is not None:
            label = f"{idx_track}: True Object" if assigns.has_assign(row=idx_track) \
                else f"{idx_track}: False Pos."
        else:
            label=f"Track {ID}"
        linestyle = "-" if "True" in label else "--"
        y = beta.pdf(x, trust_estimator.params[ID]["alpha"], trust_estimator.params[ID]["beta"])
        ax.plot(x, y, linewidth=linewidth, linestyle=linestyle, label=label)

    lines = ax.get_lines()
    xs = np.linspace(0.3, 0.7, len(lines))
    for i, line in enumerate(lines):
        labelLine(
            line,
            xs[i],
            label=r"{}".format(line.get_label().split(":")[0]),
            ha="left",
            va="bottom",
            align=False,
            backgroundcolor="none",
        )

    ax.set_xlim([0, 1.0])
    ax.set_ylim([0, 10])
    ax.legend(loc="upper left")
    ax.set_xlabel("Trust Score")
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, "experiment_trusts.pdf"))
    plt.show()


def init_trust_estimator(*args, **kwargs):
    return TrustEstimator(*args, **kwargs)


def run_metrics(trust_estimator, agents, fovs, objects):
    # assign last tracks to truths
    A = avstack.modules.assignment.build_A_from_distance(trust_estimator.tracks, objects)
    assigns = avstack.modules.assignment.gnn_single_frame_assign(A, cost_threshold=0.1)

    # # run multi-class confusion matrix
    # mean = {"params["alpha"
    # prediction = 


In [ ]:
def simulate_trust(radius=0.8, n_objects=5, frames=40, force_case=False, plot=True):
    
    # -- initialize
    agents, fovs, platforms = init_agents(radius=radius)
    objects = init_objects(fovs, n_objects=n_objects, force_object_case=force_case)
    fps, fns = init_adversary(fovs)
    tracker = init_tracker()
    trust_estimator = init_trust_estimator(do_propagate=True, propagation_model="prior")
    if plot:
        plot_agents_objects(agents, radius, objects, fps, fns)
    
    # -- run inner loop
    dt = 0.1
    for frame in range(frames):
        # detections
        timestamp = frame * dt
        dets = detect(agents, radius, objects, fps, fns, frame, timestamp)
        if plot and (frame == 0):
            plot_detections(agents, radius, dets, objects=objects)
    
        # run the trust estimator
        tracks = tracker.tracks_confirmed
        clusters = trust_estimator(agents, fovs, dets, tracks)
        if plot and (frame == 0):
            plot_clusters(agents, radius, clusters, objects=objects)
            
        # run dets through tracker
        tracker(dets, fovs=fovs, platforms=platforms, check_reference=False)
    
    # -- plot outcomes
    if plot:
        plot_tracks(agents, radius, tracker.tracks, objects=objects)
        plot_trust(trust_estimator, objects=objects)

    # -- run metrics evaluation
    metrics = run_metrics(trust_estimator, agents, fovs, objects)

    return trust_estimator, metrics

In [ ]:
trust_estimator = simulate_trust(force_case=True)
